
# Notion RAG × LangChain Companion (A/B/C Variants)

**기존 Notion 기반 RAG 커스텀 코드의 0 ~ 6단계는 유지**하고, 아래 3가지 LangChain 도입

- **A안**: 0 ~ 5은 기존 코드 그대로, **6, 7단계인 LLM 사용만 LangChain**으로 감싸기 (최소침습 어댑터)
- **B안**: 0 ~ 4은 유지, **5~7을 LangChain**으로 대체 (실험/확장 속도 강화)
- **C안**: **풀 LC** — 0 ~ 7 전체를 LangChain 생태계로 구성 (PoC/프로토타입에 유리)



In [1]:

# (선택) 필수 패키지 설치
!pip install notion-client langchain langchain-community langchain-openai chromadb tiktoken
!pip install sentence_transformers  # (선택) reranker 등 확장 시



## 가정(당신의 기존 노트북에 이미 존재하는 객체)

이 노트북의 A/B 코드들은 다음 **기존 변수/객체**가 이미 준비되어 있다고 가정합니다.

- `page_uuid`: 정규화된 Notion Page UUID (예: `"26c77d75-e008-81d5-a6ee-e7034c46c949"`)
- `markdown_text` 혹은 `raw_text`: 페이지에서 추출된 최종 텍스트(청크 분할 전)
- `chunks` 또는 `docs`: 청크 리스트 (문자열 혹은 LangChain Document 리스트 중 하나)
- `emb`: 기존 임베딩 객체(예: `OpenAIEmbeddings` 유사 API) — `.embed_query(text)` 사용 가능
- `collection`: 기존 Chroma(혹은 유사) 벡터 스토어 컬렉션 객체 — `.query(query_embeddings=[...], n_results=k)`

> 만약 변수명이 다른 경우, 아래 코드의 해당 부분만 바꿔서 사용하세요.


# 🗂️ 노션 데이터로 나만의 RAG 시스템 구축하기(기존 커스텀 RAG 코드)

노션 API로 페이지/데이터베이스를 불러와 **임베딩 → FAISS 검색 → LLM 생성**까지 한 번에 테스트하기

- **임베딩**: BAAI/bge-m3 (로컬)
- **벡터 스토어**: FAISS (메모리)
- **LLM**: OpenAI-호환 API (기본 OpenRouter; BASE_URL/MODEL_NAME 교체로 Groq/Together/Fireworks 사용 가능)

In [4]:
# 0) Install deps
!pip -q install notion-client sentence-transformers faiss-cpu openai python-dotenv

## 0) 환경 설정
- **Notion**: 내부 통합 생성 후 `NOTION_TOKEN`을 입력하세요.
    - https://www.notion.so/profile/integrations
- **LLM**: OpenRouter 기준(`PROVIDER_API_KEY` 필요). 다른 제공자는 BASE_URL/MODEL_NAME만 바꾸세요.
    - https://openrouter.ai/

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# --- Notion ---
NOTION_TOKEN = '' #'ntn_xxx'

# --- LLM (OpenAI-호환) ---
API_KEY = '' # 'sk-or-v1-xxx'
BASE_URL = "https://openrouter.ai/api/v1"
MODEL_NAME = "meta-llama/llama-3.1-8b-instruct"

# --- Embedding ---
EMB_MODEL = "BAAI/bge-m3"

print({
    "NOTION_TOKEN": bool(NOTION_TOKEN),
    "BASE_URL": BASE_URL,
    "MODEL_NAME": MODEL_NAME,
    "EMB_MODEL": EMB_MODEL,
})



{'NOTION_TOKEN': True, 'BASE_URL': 'https://openrouter.ai/api/v1', 'MODEL_NAME': 'meta-llama/llama-3.1-8b-instruct', 'EMB_MODEL': 'BAAI/bge-m3'}


## 1) Notion API 유틸 (페이지/DB → Markdown 텍스트)

In [87]:
from notion_client import Client
import re, textwrap, hashlib
from typing import List, Dict

if not NOTION_TOKEN:
    raise RuntimeError("NOTION_TOKEN이 필요합니다.")
nclient = Client(auth=NOTION_TOKEN)

def _pt(rt_list):
    return "".join([t.get("plain_text","") for t in (rt_list or [])])

def _flatten_block(block):
    t = block["type"]
    b = block[t]
    if t == "paragraph":
        return _pt(b.get("rich_text"))
    if t.endswith("_heading"):
        return "# " + _pt(b.get("rich_text"))
    if t in ("bulleted_list_item","numbered_list_item","to_do"):
        return "- " + _pt(b.get("rich_text"))
    if t == "quote":
        return "> " + _pt(b.get("rich_text"))
    if t == "code":
        txt = b.get("rich_text", [{}])[0].get("plain_text","")
        lang = b.get("language","")
        return f"```{lang}\n"+txt+"\n```"
    if t == "callout":
        return "💡 " + _pt(b.get("rich_text"))
    if t == "toggle":
        return _pt(b.get("rich_text"))  # children로 확장
    if t == "equation":
        return "$" + b.get("expression","") + "$"
    if t == "table_row":
        cells = [ _pt(cell) for cell in b.get("cells", []) ]
        return " | ".join(cells)
    return ""

def _walk_children(block_id, acc: List[str]):
    children = nclient.blocks.children.list(block_id=block_id)
    while True:
        for b in children["results"]:
            acc.append(_flatten_block(b))
            if b.get("has_children"):
                _walk_children(b["id"], acc)
        if not children.get("has_more"): break
        children = nclient.blocks.children.list(block_id=block_id, start_cursor=children["next_cursor"])

# 페이지를 재귀로 순회해 텍스트화
def notion_page_to_markdown(page_id: str) -> str:
    out=[]
    _walk_children(page_id, out)
    md = "\n".join(filter(None,out)).strip()
    return md

def get_page_meta(page: Dict) -> Dict:
    # title property 찾기
    props = page.get("properties", {})
    title_prop = next((v for v in props.values() if v.get("type")=="title"), None)
    title = _pt((title_prop or {}).get("title", [])) or page.get("id")
    return {
        "page_id": page["id"],
        "title": title,
        "url": page.get("url"),
        "last_edited_time": page.get("last_edited_time"),
    }

# DB의 각 페이지를 위 함수로 변환
def fetch_pages_from_database(database_id: str) -> List[Dict]:
    results=[]
    resp = nclient.databases.query(database_id=database_id, page_size=50)
    while True:
        for page in resp["results"]:
            meta = get_page_meta(page)
            md = notion_page_to_markdown(page["id"])
            results.append({**meta, "content_md": md})
        if not resp.get("has_more"): break
        resp = nclient.databases.query(database_id=database_id, page_size=50, start_cursor=resp["next_cursor"])
    return results

# 단일 페이지 변환
def fetch_single_page(page_id: str) -> Dict:
    page = nclient.pages.retrieve(page_id=page_id)
    meta = get_page_meta(page)
    md = notion_page_to_markdown(page_id)
    return {**meta, "content_md": md}

## 2) 대상 선택: 데이터베이스 ID 또는 개별 페이지 ID
- 원하는 `database_id`/`page_id`를 넣어주세요.

In [90]:
# 예시: 하나의 데이터베이스를 긁어오거나, 개별 페이지들을 모아올 수 있습니다.
DATABASE_IDS = [
    # "264bf0ad3a0680e18fedda127323e553",
    # "15c8cc2b57a44ae7901d48122862a22a",
]
PAGE_IDS = [
    "24977d75e00880369b93e2653613eec3",
    # "44ce9a98e0074495922c917ba96cf631",
    "22e77d75e00880ea9085d1410be465fb",
    # "15c8cc2b57a44ae7901d48122862a22a",
]

## 3) Notion → 문서 리스트 로드

In [91]:
docs = []

for dbid in DATABASE_IDS:
    docs += fetch_pages_from_database(dbid)

for pid in PAGE_IDS:
    docs.append(fetch_single_page(pid))

len(docs), [ (d['title'], d['url']) for d in docs[:5] ]

(2,
 [('스터디방향성', 'https://www.notion.so/24977d75e00880369b93e2653613eec3'),
  ('계획', 'https://www.notion.so/22e77d75e00880ea9085d1410be465fb')])

In [92]:
docs

[{'page_id': '24977d75-e008-8036-9b93-e2653613eec3',
  'title': '스터디방향성',
  'url': 'https://www.notion.so/24977d75e00880369b93e2653613eec3',
  'last_edited_time': '2025-08-24T04:13:00.000Z',
  'content_md': '카테고리 | 많이 쓰는 도구 | 이유 / 현업 사용 포인트\n데이터 버전관리 | DVC, Git LFS | DVC는 데이터 파이프라인 추적 + 원격 저장 연동에 강함, Git LFS는 단순 대용량 파일 버전 관리\n파이프라인 관리 | Airflow, Prefect | Airflow는 대규모 기업·배치 작업 표준, Prefect는 파이썬 친화적이고 설정이 단순\n실험 관리 | MLflow, wandb | MLflow는 오픈소스 표준, wandb는 시각화·협업 기능이 강하고 스타트업/연구실에서 인기\n분산 학습 | PyTorch DDP, Ray | 대규모 학습은 DDP가 표준, Ray는 DDP 래핑 + Task 분산 둘 다 가능\n모델 서빙 | FastAPI, BentoML | FastAPI는 가볍고 유연, BentoML은 모델 버전 관리 + 서빙 일체형\n배포 | Docker, Kubernetes(K8s) | Docker는 개발·테스트 필수, 운영 환경 확장은 K8s가 사실상 표준\n모니터링 | Prometheus + Grafana, Evidently AI | Prom+Graf는 시스템/모델 지표 모니터링 표준, Evidently는 데이터 드리프트·성능 모니터링 특화\n- DHKim역량: 실험 관리, 분산학습, 배포, 모니터링 쪽 가능 \n- 실험관리 (visualization): MLflow, wandb\n- 분산학습: Ray, PyTorchDDP, Accelerate 등\n- 배포: Docker\n- 모니터링: Prometheus + Grafana (현재 연구실 GPU모니터링 시스템 만든이력있

## 4) 전처리 & 청킹(Chunking)

In [93]:
def split_markdown(md: str, max_len=900):
    parts=[]; buf=[]
    for line in md.splitlines():
        if re.match(r"^#{1,6}\s", line) and buf:
            chunk="\n".join(buf).strip()
            parts += textwrap.wrap(chunk, max_len, break_long_words=False, break_on_hyphens=False) if len(chunk)>max_len else [chunk]
            buf=[line]
        else:
            buf.append(line)
    if buf:
        chunk="\n".join(buf).strip()
        parts += textwrap.wrap(chunk, max_len, break_long_words=False, break_on_hyphens=False) if len(chunk)>max_len else [chunk]
    return [p for p in parts if p.strip()]

chunks=[]
metas=[]
for d in docs:
    for ch in split_markdown(d["content_md"]):
        metas.append({"page_id": d["page_id"], "title": d["title"], "url": d.get("url"), "section": "", "text": ch})
        chunks.append(ch)

## 5) 임베딩 & 벡터 인덱스(FAISS)

In [94]:
from sentence_transformers import SentenceTransformer

e_model = SentenceTransformer(EMB_MODEL)

def embed(texts):
    return e_model.encode(texts, normalize_embeddings=True, convert_to_numpy=True).astype("float32")

vecs = embed(chunks)

In [95]:
import numpy as np, faiss

class FaissStore:
    def __init__(self, dim):
        self.index = faiss.IndexFlatIP(dim)
        self.meta = []
    def add(self, vecs, metas):
        self.index.add(vecs)    # 학습 불필요, 바로 추가
        self.meta += metas
    def search(self, qvec, k=5):
        D,I = self.index.search(np.array([qvec]).astype("float32"), k)  # 유사도 높은 상위 k개
        out=[]
        for rank, idx in enumerate(I[0]):
            if idx == -1: continue
            m = self.meta[idx]
            out.append({"text": m["text"], "meta": {k:v for k,v in m.items() if k!="text"}, "score": float(D[0][rank])})
        return out

store = FaissStore(vecs.shape[1])
store.add(vecs, metas)
len(chunks)

9

## 6) LLM 호출 (OpenAI-호환) 및 질의 → 검색 → 답변
- 기본 OpenRouter, 필요시 BASE_URL/MODEL_NAME 교체

In [124]:
from openai import OpenAI

if not API_KEY:
    raise RuntimeError("PROVIDER_API_KEY가 필요합니다.")

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
SYSTEM = "당신은 신뢰 가능한 한국어 어시스턴트입니다. 제공된 근거 외 추측 금지."

def build_prompt(query, contexts):
    ctx = "\n\n---\n\n".join(
        f"[{i+1}] {c['meta'].get('title','(제목없음)')} / {c['meta'].get('section','')}\n{c['text']}"
        for i,c in enumerate(contexts)
    )
    return f"""사용자 질문: {query}

다음 근거를 바탕으로 한국어로 정확히 답하세요.
근거:
{ctx}

규칙:
- 근거에 없는 내용은 '근거 없음'으로 표시
- 필요한 경우 목록/표로 간결히
- 각 주장에는 근거 번호를 붙여라
"""

def llm_answer(query, contexts, temperature=0.2, max_tokens=800):
    prompt = build_prompt(query, contexts)
    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role":"system","content":SYSTEM}, {"role":"user","content":prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return resp.choices[0].message.content


def embed_one(text):
    return embed([text])[0]

def ask(q: str, k: int = 8, n_ctx: int = 5):
    qv = embed_one(q)
    cands = store.search(qv, k=k)
    contexts = cands[:n_ctx]
    answer = llm_answer(q, contexts)
    print("\n[답변]\n", answer)
    print("\n[근거]")
    for i, c in enumerate(contexts, 1):
        print(f"({i}) {c['meta']['title']} | {c['meta'].get('url','')}")
    return answer, contexts

# 예시 실행:
answer, ctx = ask("Week 3 계획")



[답변]
 Week 3 계획은 다음과 같습니다.

- 로또의 최고 순위와 최저 순위 (Lv.1)
- 신고 결과 받기 (Lv.1)
- 명예의 전당 (1) (Lv.1)
- 개인정보 수집 유효기간 (Lv.1~2)
- 햄버거 만들기 (Lv.1)

이 계획은 조건 + 구현 + 시뮬레이션 핵심 개념을 학습하는 주차입니다.

[근거]
(1) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(2) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(3) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(4) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3
(5) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3



---
# A안: 6단계만 LangChain으로 감싸기 (최소한만 LangChain으로 변환)

- 0)~5)까지는 **기존 코드 그대로 사용**
- 6) **프롬프트→LLM** 부분만 LangChain 체인으로 선언적으로 구성

In [125]:
# --- LangChain 버전: lanchain을 사용하기 위한 구조로 변형하는 작업 필요!! ---
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
# StrOutputParser 경로 호환
try:
    from langchain.schema.output_parser import StrOutputParser
except ImportError:
    from langchain_core.output_parsers import StrOutputParser

_prompt_LC = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("user",
     "사용자 질문: {query}\n\n"
     "다음 근거를 바탕으로 한국어로 정확히 답하세요.\n"
     "근거:\n{contexts}\n\n"
     "규칙:\n"
     "- 근거에 없는 내용은 '근거 없음'으로 표시\n"
     "- 필요한 경우 목록/표로 간결히\n"
     "- 각 주장에는 근거 번호를 붙여라")
])


# LangChain 호환 가능 OpenAI 불러오기 ** LangChain 사용 가능한 모델?만 가능함 / 제약이 있는편(그래도 대부분 모델 가능)
# LLM 정의 (기존 client와 동일한 설정)
try:
    LC_llm = ChatOpenAI(
        model=MODEL_NAME,
        api_key=API_KEY,
        base_url=BASE_URL,
    )
except TypeError:  # 버전 호환
    LC_llm = ChatOpenAI(
        model=MODEL_NAME,
        openai_api_key=API_KEY,
        openai_api_base=BASE_URL,
    )

# 체인: build_prompt → LLM → 파서
chain_A = (
    RunnableLambda(lambda x: build_prompt(x["query"], x["contexts"]))
    | LC_llm
    | StrOutputParser() # 문자열만 그대로 Output내는 함수
)

def _format_contexts(contexts):
    blocks = []
    for i, c in enumerate(contexts, 1):
        meta = c.get("meta", {}) or {}
        title = meta.get("title", "(제목없음)")
        section = meta.get("section", "")
        text = c.get("text", "")
        blocks.append(f"[{i}] {title} / {section}\n{text}")
    return "\n\n---\n\n".join(blocks)

# 함수: ask_A에서 쓸 수 있게 래핑
def llm_answer_A(query, contexts, temperature=0.2, max_tokens=800):
    llm_bound = LC_llm.bind(temperature=temperature,
                           max_tokens=max_tokens,
                           max_completion_tokens=max_tokens)
    # 검색을 위한 프롬프트 생성부터 LLM 모델로 문장 생성까지 Chain으로 연결
    chain = (
        {  # 입력 매핑
            "query": RunnablePassthrough(),
            "contexts": RunnableLambda(lambda x: _format_contexts(x["contexts"]))
        }
        | _prompt_LC      # ← 여기서 system/user 메시지로 변환
        | llm_bound
        | StrOutputParser()
    )
    return chain.invoke({"query": query, "contexts": contexts})

def embed_one(text):
    return embed([text])[0]

def ask_A(q: str, k: int = 8, n_ctx: int = 5):
    qv = embed_one(q)
    cands = store.search(qv, k=k) # 검색 단계는 기존 함수 유지
    contexts = cands[:n_ctx]
    answer = llm_answer_A(q, contexts)
    print("\n[답변]\n", answer)
    print("\n[근거]")
    for i, c in enumerate(contexts, 1):
        print(f"({i}) {c['meta']['title']} | {c['meta'].get('url','')}")
    return answer, contexts

# 예시 실행:
answer, ctx = ask_A("Week 3 계획")


[답변]
 Week 3 계획에 대한 정보는 다음과 같습니다.

- Week 3 – 조건 + 구현 + 시뮬레이션 핵심 개념: 조건 분기문, 값 누적, 반복문 흐름 설계
- 핵심 개념에 포함된 문제는 다음과 같습니다.
  - 로또의 최고 순위와 최저 순위 (Lv.1)
  - 신고 결과 받기 (Lv.1)
  - 명예의 전당 (1) (Lv.1)
  - 개인정보 수집 유효기간 (Lv.1~2)
  - 햄버거 만들기 (Lv.1)

[근거]
(1) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(2) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(3) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(4) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3
(5) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3



---
# B안: 5~6단계 LangChain 교체

- 5)~6) **임베딩/벡터인덱스 + QA 체인(A안 6 단계)** 부분을 LangChain 체인으로 구성

In [126]:
# --- 호환 임포트 ---
try:
    from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
except ImportError:
    from langchain_core.runnables import RunnableLambda, RunnablePassthrough

try:
    from langchain.schema.output_parser import StrOutputParser
except ImportError:
    from langchain_core.output_parsers import StrOutputParser

# --- 벡터스토어 / 임베딩 래퍼 ---
try:
    from langchain_core.embeddings import Embeddings
except ImportError:
    from langchain.embeddings.base import Embeddings

class LegacyEmbeddings(Embeddings):
    def __init__(self, embed_fn): self.embed_fn = embed_fn
    def embed_documents(self, texts): return self.embed_fn(texts)
    def embed_query(self, text): return self.embed_fn([text])[0]

from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.docstore.document import Document as LCDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# ===== 0) SYSTEM 분리 프롬프트 =====
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

_prompt_LC = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("user",
     "사용자 질문: {query}\n\n"
     "다음 근거를 바탕으로 한국어로 정확히 답하세요.\n"
     "근거:\n{contexts}\n\n"
     "규칙:\n"
     "- 근거에 없는 내용은 '근거 없음'으로 표시\n"
     "- 필요한 경우 목록/표로 간결히\n"
     "- 각 주장에는 근거 번호를 붙여라")
])

# LLM 초기화 (기존 모델/키/base_url 그대로)
try:
    LC_llm = ChatOpenAI(model=MODEL_NAME, api_key=API_KEY, base_url=BASE_URL)
except TypeError:
    LC_llm = ChatOpenAI(model=MODEL_NAME, openai_api_key=API_KEY, openai_api_base=BASE_URL)

# ===== 1) 제목 보완(헤더 추정) =====
def _safe_title(meta: dict, text: str):
    t = (meta or {}).get("title")
    if isinstance(t, str) and t.strip():
        return t.strip()
    m = re.search(r"^\s*#{1,6}\s+(.+)$", text, flags=re.MULTILINE)
    return (m.group(1).strip() if m else "(제목없음)")

# ===== 2) docs(LangChain Document or Notion dict) → LangChain Document 표준화 =====
# FAISS 등 LangChain의 VectorStore는 반드시 Document 타입을 입력으로 받음
def _to_documents(docs_or_chunks):
    docs = []
    for item in docs_or_chunks:
        if isinstance(item, LCDocument):
            meta = dict(item.metadata or {})
            if "title" not in meta:
                meta["title"] = _safe_title(meta, item.page_content)
            if "url" not in meta:
                pid = meta.get("page_id") or meta.get("id")
                if isinstance(pid, str) and pid:
                    meta["url"] = f"https://www.notion.so/{pid.replace('-','')}"
            docs.append(Document(page_content=item.page_content, metadata=meta))
        elif isinstance(item, dict):
            text = item.get("content_md") or item.get("text") or ""
            meta = {
                "title": item.get("title") or "",
                "url": item.get("url") or "",
                "page_id": item.get("page_id") or item.get("id"),
                "last_edited_time": item.get("last_edited_time"),
            }
            if not meta["title"]:
                meta["title"] = _safe_title(meta, text)
            if not meta["url"] and meta.get("page_id"):
                meta["url"] = f"https://www.notion.so/{meta['page_id'].replace('-','')}"
            docs.append(Document(page_content=text, metadata=meta))
        else:
            docs.append(Document(page_content=str(item), metadata={}))
    return docs

# ===== 3) 이미 있는 docs로 바로 retriever =====
def build_retriever_B_from_docs(docs_ready, k: int = 8):
    emb = LegacyEmbeddings(embed)   # 기존 embed([...]) 재사용
    docs_std = _to_documents(docs_ready)
    vs = FAISS.from_documents(docs_std, emb)
    return vs.as_retriever(search_kwargs={"k": k})

# ===== 4) LangChain Document → 기존 컨텍스트(dict) 포맷 + title/url 보장 =====
def _contexts_from_docs(docs):
    ctxs = []
    for d in docs:
        meta = dict(d.metadata or {})
        if "title" not in meta or not meta["title"]:
            meta["title"] = _safe_title(meta, d.page_content)
        if "url" not in meta or not meta["url"]:
            pid = meta.get("page_id") or meta.get("id")
            if isinstance(pid, str) and pid:
                meta["url"] = f"https://www.notion.so/{pid.replace('-','')}"
        ctxs.append({"text": d.page_content, "meta": meta})
    return ctxs

# ===== 5) contexts(dict 리스트) → 프롬프트용 문자열 =====
def _format_contexts_for_prompt(contexts):
    blocks = []
    for i, c in enumerate(contexts, 1):
        meta = c.get("meta", {}) or {}
        title = meta.get("title", "(제목없음)")
        section = meta.get("section", "")
        text = c.get("text", "")
        blocks.append(f"[{i}] {title} / {section}\n{text}")
    return "\n\n---\n\n".join(blocks)

# ===== 6) 체인 빌더: retriever → (query,contexts_str) → _prompt_LC → LLM → Parser =====
def llm_answer_B(retriever, *, temperature: float = 0.2, max_tokens: int = 800):
    llm_bound = LC_llm.bind(
        temperature=temperature,
        max_tokens=max_tokens,
        max_completion_tokens=max_tokens
    )
    chain = (
        {"query": RunnablePassthrough(), "docs": retriever}   # 질의 → 검색
        | RunnableLambda(lambda x: {
            "query": x["query"],
            "contexts": _format_contexts_for_prompt(_contexts_from_docs(x["docs"]))
        })
        | _prompt_LC
        | llm_bound
        | StrOutputParser()
    )
    return chain

# ===== 7) 실행 헬퍼 =====
def ask_B(q: str, chain, retriever, n_ctx: int = 5):
    answer = chain.invoke({"query": q})
    docs_top = retriever.get_relevant_documents(q)[:n_ctx]
    contexts = _contexts_from_docs(docs_top)

    print("\n[답변]\n", answer)
    print("\n[근거]")
    for i, c in enumerate(contexts, 1):
        print(f"({i}) {c['meta'].get('title','(제목없음)')} | {c['meta'].get('url','')}")
    return answer, contexts

# ===== 8) 사용 예시 =====
retrieverB = build_retriever_B_from_docs(docs, k=8)  # docs: 당신이 가진 Notion dict 리스트
chainB = llm_answer_B(retrieverB, temperature=0.2, max_tokens=800)
question = "Week 3 계획"
answer, ctx = ask_B(question, chainB, retrieverB, n_ctx=5)



[답변]
 Week 3 계획입니다.

[근거]
(1) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(2) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3



---
# C안: Full LangChain 파이프라인

- 데이터 로딩(~docs 생성까지 유지) -> 청킹 -> 임베딩/인덱스 -> 검색(리트리버) -> QA 체인

In [ ]:
# LangChain의 Notion 데이터 가져오는 라이브러리가 있는데 데이터베이스만 가능,,,

In [138]:
# ===============================
# C안: Full LangChain 파이프라인
# 로딩(선택) → 청킹 → 임베딩/인덱스 → Retriever → SYSTEM 분리 프롬프트 → LLM → 파서
# ===============================

# ---- 호환 임포트 (버전별 경로 차이 처리) ----
try:
    from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
except ImportError:
    from langchain_core.runnables import RunnableLambda, RunnablePassthrough

try:
    from langchain.schema.output_parser import StrOutputParser
except ImportError:
    from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS

try:
    from langchain_core.embeddings import Embeddings
except ImportError:
    from langchain.embeddings.base import Embeddings

import re, uuid
from typing import List, Iterable, Dict, Any

# ---- LLM 초기화 (기존 설정 그대로) ----
try:
    LC_llm = ChatOpenAI(model=MODEL_NAME, api_key=API_KEY, base_url=BASE_URL)
except TypeError:
    LC_llm = ChatOpenAI(model=MODEL_NAME, openai_api_key=API_KEY, openai_api_base=BASE_URL)

# ---- SYSTEM 분리 프롬프트 (기존 규칙 유지) ----
_prompt_C = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("user",
     "사용자 질문: {query}\n\n"
     "다음 근거를 바탕으로 한국어로 정확히 답하세요.\n"
     "근거:\n{contexts}\n\n"
     "규칙:\n"
     "- 근거에 없는 내용은 '근거 없음'으로 표시\n"
     "- 필요한 경우 목록/표로 간결히\n"
     "- 각 주장에는 근거 번호를 붙여라")
])

# ---- 기존 embed([...])를 LangChain Embeddings로 래핑 ----
class LegacyEmbeddings(Embeddings):
    def __init__(self, embed_fn): self.embed_fn = embed_fn
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.embed_fn(texts)
    def embed_query(self, text: str) -> List[float]:
        return self.embed_fn([text])[0]

# ---- 제목 보완 (헤더에서 추정) ----
def _safe_title(meta: dict, text: str):
    t = (meta or {}).get("title")
    if isinstance(t, str) and t.strip():
        return t.strip()
    m = re.search(r"^\s*#{1,6}\s+(.+)$", text, flags=re.MULTILINE)
    return (m.group(1).strip() if m else "(제목없음)")

# ---- Notion dict → LangChain Document 변환 (docs가 이미 있으면 이걸 사용) ----
def docs_from_notion_dicts(items: List[dict]) -> List[Document]:
    out = []
    for it in items:
        text = it.get("content_md") or it.get("text") or ""
        meta = {
            "title": it.get("title") or "",
            "url": it.get("url") or "",
            "page_id": it.get("page_id") or it.get("id"),
            "last_edited_time": it.get("last_edited_time"),
            "section": it.get("section",""),
        }
        if not meta["title"]:
            meta["title"] = _safe_title(meta, text)
        if not meta["url"] and isinstance(meta.get("page_id"), str) and meta["page_id"]:
            meta["url"] = f"https://www.notion.so/{meta['page_id'].replace('-','')}"
        out.append(Document(page_content=text, metadata=meta))
    return out

# ---- (선택) Notion DB에서 직접 로딩하려면 사용 ----
# from langchain_community.document_loaders import NotionDBLoader
# def _normalize_notion_id(raw: str, with_hyphens: bool = True) -> str:
#     hex32 = "".join(re.findall(r"[0-9a-fA-F]", raw)).lower()
#     if len(hex32) != 32: raise ValueError(f"Notion ID 길이 오류: {raw}")
#     return str(uuid.UUID(hex32)) if with_hyphens else hex32
# def load_notion_db_docs(database_id: str, notion_token: str) -> List[Document]:
#     db_id = _normalize_notion_id(database_id, with_hyphens=True)
#     loader = NotionDBLoader(integration_token=notion_token, database_id=db_id)
#     docs_loaded = loader.load()
#     out = []
#     for d in docs_loaded:
#         meta = dict(d.metadata or {})
#         if "title" not in meta or not meta["title"]:
#             m = re.search(r"^\s*#{1,6}\s+(.+)$", d.page_content, flags=re.MULTILINE)
#             meta["title"] = (m.group(1).strip() if m else "(제목없음)")
#         if "url" not in meta or not meta["url"]:
#             pid = meta.get("page_id") or meta.get("id")
#             if isinstance(pid, str) and pid:
#                 meta["url"] = f"https://www.notion.so/{pid.replace('-','')}"
#         out.append(Document(page_content=d.page_content, metadata=meta))
#     return out

# ---- 청킹 ----
def chunk_documents(docs: List[Document], chunk_size: int = 1000, chunk_overlap: int = 150) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(docs)

# ---- 인덱싱/리트리버 (FAISS) ----
def build_vs_and_retriever_C(chunked_docs: List[Document],
                             embed_like=None,
                             k: int = 8,
                             search_type: str = "mmr",
                             search_kwargs: Dict[str, Any] = None):
    embeddings = embed_like or LegacyEmbeddings(embed)  # embed([...]) 사용
    vs = FAISS.from_documents(chunked_docs, embeddings)
    if search_kwargs is None:
        search_kwargs = {"k": k, "fetch_k": 32, "lambda_mult": 0.5} if search_type=="mmr" else {"k": k}
    retriever = vs.as_retriever(search_type=search_type, search_kwargs=search_kwargs)
    return vs, retriever

# ---- 컨텍스트 변환 (LLM 프롬프트용 문자열) ----
def _contexts_from_docs(docs: Iterable[Document]):
    ctxs = []
    for d in docs:
        meta = dict(d.metadata or {})
        if "title" not in meta or not meta["title"]:
            meta["title"] = _safe_title(meta, d.page_content)
        if "url" not in meta or not meta["url"]:
            pid = meta.get("page_id") or meta.get("id")
            if isinstance(pid, str) and pid:
                meta["url"] = f"https://www.notion.so/{pid.replace('-','')}"
        ctxs.append({"text": d.page_content, "meta": meta})
    return ctxs

def _format_contexts_for_prompt(contexts: List[Dict[str, Any]]) -> str:
    blocks = []
    for i, c in enumerate(contexts, 1):
        meta = c.get("meta", {}) or {}
        title = meta.get("title", "(제목없음)")
        section = meta.get("section", "")
        text = c.get("text", "")
        blocks.append(f"[{i}] {title} / {section}\n{text}")
    return "\n\n---\n\n".join(blocks)

# ---- QA 체인 (Retriever → Prompt → LLM → Parser) ----
def build_chain_C(retriever, *, temperature: float = 0.2, max_tokens: int = 800):
    llm_bound = LC_llm.bind(temperature=temperature, max_tokens=max_tokens, max_completion_tokens=max_tokens)
    chain = (
        {"query": RunnablePassthrough(), "docs": retriever}
        | RunnableLambda(lambda x: {
            "query": x["query"],
            "contexts": _format_contexts_for_prompt(_contexts_from_docs(x["docs"]))
        })
        | _prompt_C
        | llm_bound
        | StrOutputParser()
    )
    return chain

def ask_C(question: str, chain, retriever, n_ctx: int = 5):
    """실행 + 상위 근거 출력 + 컨텍스트 반환"""
    answer = chain.invoke({"query": question})
    top_docs = retriever.get_relevant_documents(question)[:n_ctx]
    contexts = _contexts_from_docs(top_docs)

    print("\n[답변]\n", answer)
    print("\n[근거]")
    for i, c in enumerate(contexts, 1):
        print(f"({i}) {c['meta'].get('title','(제목없음)')} | {c['meta'].get('url','')}")
    return answer, contexts

# ===============================
# 실행 예시
# ===============================
# (A) 이미 가지고 있는 docs(dict 리스트)를 사용
docs_lc = docs_from_notion_dicts(docs)           # dict → LC Document
chunked = chunk_documents(docs_lc, 1000, 150)    # 청킹
vsC, retrieverC = build_vs_and_retriever_C(chunked, k=8, search_type="mmr")
chainC = build_chain_C(retrieverC, temperature=0.2, max_tokens=800)

question = "Week 3 계획"
answer, ctx = ask_C(question, chainC, retrieverC, n_ctx=5)

# (B) 다른 임베딩 쓰고 싶으면:
# from langchain_openai import OpenAIEmbeddings
# emb_alt = OpenAIEmbeddings(model="text-embedding-3-large", api_key=API_KEY, base_url=BASE_URL)
# vsC2, retrieverC2 = build_vs_and_retriever_C(chunked, embed_like=emb_alt, k=8, search_type="similarity")
# chainC2 = build_chain_C(retrieverC2)
# answer2, ctx2 = ask_C("질문", chainC2, retrieverC2)



[답변]
 Week 3 계획입니다.

Week 3에서는 조건 + 구현 + 시뮬레이션을 주제로 공부합니다. 핵심 개념은 조건 분기문, 값 누적, 반복문 흐름 설계입니다.

주어진 문제는 다음과 같습니다.

- 로또의 최고 순위와 최저 순위 (Lv.1)
- 신고 결과 받기 (Lv.1)
- 명예의 전당 (1) (Lv.1)
- 개인정보 수집 유효기간 (Lv.1~2)
- 햄버거 만들기 (Lv.1)

이 문제들은 조건 분기문, 값 누적, 반복문 흐름 설계를 사용하여 해결할 수 있습니다.

[근거]
(1) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(2) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3
(3) 계획 | https://www.notion.so/22e77d75e00880ea9085d1410be465fb
(4) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3
(5) 스터디방향성 | https://www.notion.so/24977d75e00880369b93e2653613eec3
